##Project 4: Exploring the UK's milk imports and exports

by Tony Hirst and Michel Wermelinger, 19 November 2015,
extended by Ricardo Romo Encino, January 1st 2016

This is the project notebook for Week 4 of The Open University's [_Learn to code for Data Analysis_](http://futurelearn.com/courses/learn-to-code) course.

A country's economy depends, sometimes heavily, on its exports and imports. The United Nations Comtrade database provides data on global trade. It will be used to analyse the UK's imports and exports of milk and cream in 2015:

- How much does the UK export and import and is the balance positive (more exports than imports)? 
- Which are the main trading partners, i.e. from/to which countries does the UK import/export the most?
- Which are the regular customers, i.e. which countries buy milk from the UK every month?
- Which countries does the UK both import from and export to?

In [2]:
from pandas import *

##Getting and preparing the data

The data is obtained from the [United Nations Comtrade](http://comtrade.un.org/data/) website, by selecting the following configuration:

- Frequency: monthly 
- Periods: January to May of 2015
- Reporter: United Kingdom
- Partners: all
- Flows: imports and exports
- HS (as reported) commodity codes: 0401 (Milk and cream, neither concentrated nor sweetened) and 0402 (Milk and cream, concentrated or sweetened)

Clicking on 'Preview' results in a message that the data exceeds 500 rows and reveals a 'View API call' link at the bottom of the page. Clicking on it shows the URL, which has to be changed in two ways: `max=500` is increased to `max=5000` to make sure all data is loaded, and `&fmt=csv` is added at the end to obtain the data in CSV format. 

In addition, the commodity code has to be read as a string, to not lose the leading zero.

In [3]:
URL = 'http://comtrade.un.org/api/get?max=5000&type=C&freq=M&px=HS&ps=201505%2C201504%2C201503%2C201502%2C201501&r=826&p=all&rg=1%2C2&cc=0401%2C0402&fmt=csv'
milk = read_csv(URL, dtype={'Commodity Code':str})
milk.tail(2)

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Qty,Alt Qty Unit Code,Alt Qty Unit,Alt Qty,Netweight (kg),Gross weight (kg),Trade Value (US$),CIF Trade Value (US$),FOB Trade Value (US$),Flag
633,HS,2015,201505,May 2015,4,0,2,Exports,826,United Kingdom,...,NaN,NaN,NaN,NaN,2213,NaN,37883,NaN,NaN,0
634,HS,2015,201505,May 2015,4,0,2,Exports,826,United Kingdom,...,NaN,NaN,NaN,NaN,1588,NaN,5676,NaN,NaN,0


The data only covers the first five months. Most columns are irrelevant for this analysis, or contain always the same value, like the year and reporter columns. The commodity code is transformed into a short but descriptive text and only the relevant columns are selected.

In [4]:
def milkType(code):
    if code == '0401': # neither concentrated nor sweetened
        return 'unprocessed'
    if code == '0402': # concentrated or sweetened
        return 'processed' 
    return 'unknown'

COMMODITY = 'Milk and cream'
milk[COMMODITY] = milk['Commodity Code'].apply(milkType)
MONTH = 'Period'
PARTNER = 'Partner'
FLOW = 'Trade Flow'
VALUE = 'Trade Value (US$)'
headings = [MONTH, PARTNER, FLOW, COMMODITY, VALUE]
milk = milk[headings]
milk.head()

,Period,Partner,Trade Flow,Milk and cream,Trade Value (US$)
0,201501,World,Imports,unprocessed,13934595
1,201501,World,Exports,unprocessed,26259775
2,201501,Australia,Exports,unprocessed,50331
3,201501,Austria,Exports,unprocessed,360
4,201501,Belgium,Imports,unprocessed,1424271


The data contains the total imports and exports per month, under the 'World' partner. Those rows are removed to keep only the per-country data.

In [5]:
milk = milk[milk[PARTNER] != 'World']
milk.head()

,Period,Partner,Trade Flow,Milk and cream,Trade Value (US$)
2,201501,Australia,Exports,unprocessed,50331
3,201501,Austria,Exports,unprocessed,360
4,201501,Belgium,Imports,unprocessed,1424271
5,201501,Belgium,Exports,unprocessed,996031
6,201501,Bulgaria,Exports,unprocessed,191


## Total trade flow

To answer the first question, 'how much does the UK export and import and is the balance positive (more exports than imports)?',
the dataframe is split into two groups: exports from the UK and imports into the UK. The  trade values within each group are summed up to get the total trading.

In [6]:
grouped = milk.groupby([FLOW])
grouped[VALUE].aggregate(sum)

Trade Flow
Exports    261466565
Imports    155907419
Name: Trade Value (US$), dtype: int64

This shows a trade surplus of over 100 million dollars.

## Main trade partners

To address the second question, 'Which are the main trading partners, i.e. from/to which countries does the UK import/export the most?', the dataframe is split by country instead, and then each group aggregated for the total trade value. This is done separately for imports and exports. The result is sorted in descending order so that the main partners are at the top.

In [7]:
imports = milk[milk[FLOW] == 'Imports']
grouped = imports.groupby([PARTNER])
print('The UK imports from', len(grouped), 'countries.')
print('The 5 biggest exporters to the UK are:')
grouped[VALUE].aggregate(sum).sort(inplace=False,ascending=False).head()

The UK imports from 21 countries.
The 5 biggest exporters to the UK are:


Partner
Ireland        46161883
France         28035441
Germany        21895561
Netherlands    17658912
Belgium        14325697
Name: Trade Value (US$), dtype: int64

In [8]:
exports = milk[milk[FLOW] == 'Exports']
grouped = exports.groupby([PARTNER])
print('The UK exports to', len(grouped), 'countries.')
print('The 5 biggest importers from the UK are:')
grouped[VALUE].aggregate(sum).order(ascending=False).head()

The UK exports to 107 countries.
The 5 biggest importers from the UK are:


Partner
Ireland                 127401424
Netherlands              17605174
China                    14111163
China, Hong Kong SAR      9482458
France                    9371510
Name: Trade Value (US$), dtype: int64

## Regular importers

Given that there are two commodities, the third question, 'Which are the regular customers, i.e. which countries buy milk from the UK every month?', is meant in the sense that a regular customer imports both commodities every month. This means that if the exports dataframe is grouped by country, each group has exactly ten rows (two commodities bought each of the five months). To see the countries, only the first month of one commodity has to be listed, as by definition it's the same countries every month and for the other commodity.

In [9]:
def buysEveryMonth(group):
    return len(group) == 10

grouped = exports.groupby([PARTNER])
regular = grouped.filter(buysEveryMonth)
regular[(regular[MONTH] == 201501) & (regular[COMMODITY] == 'processed')]

,Period,Partner,Trade Flow,Milk and cream,Trade Value (US$)
267,201501,Belgium,Exports,processed,141985
270,201501,China,Exports,processed,1013141
274,201501,Cyprus,Exports,processed,6625
279,201501,Denmark,Exports,processed,107883
283,201501,France,Exports,processed,178262
286,201501,Germany,Exports,processed,207480
291,201501,"China, Hong Kong SAR",Exports,processed,1110086
293,201501,Hungary,Exports,processed,2119
297,201501,Ireland,Exports,processed,4019355
299,201501,Italy,Exports,processed,23267


Just over 75% of the total UK exports are due to these regular customers.

In [10]:
regular[VALUE].sum() / exports[VALUE].sum()

0.7546521101082274

## Bi-directional trade

To address the fourth question, 
'Which countries does the UK both import from and export to?', a pivot table is used to list the total export and import value for each country. 

In [11]:
countries = pivot_table(milk, index=[PARTNER], columns=[FLOW], 
                        values=VALUE, aggfunc=sum)
countries.head()

Trade Flow,Exports,Imports
Partner,,
Algeria,8087885,NaN
Angola,2416129,NaN
Antigua and Barbuda,22218,NaN
"Areas, nes",51205,NaN
Australia,81644,NaN


Removing the rows with a missing value will result in only those countries with bi-directional trade flow with the UK.

In [12]:
countries.dropna()

Trade Flow,Exports,Imports
Partner,,
Austria,56279,780
Belgium,7206795,14325697
Czech Rep.,2590,486628
Denmark,335068,13681759
France,9371510,28035441
Germany,8446290,21895561
Hungary,71378,4762
Ireland,127401424,46161883
Italy,173687,315677


## Extended Project

Which are the regular exporters, i.e. which countries sell every month both unprocessed and processed milk and cream to the UK?

In [13]:
#def buysEveryMonth(group):
#    return len(group) == 10

groupedExporters = imports.groupby([PARTNER])
regularExporters = groupedExporters.filter(buysEveryMonth)
regularExporters[(regularExporters[MONTH] == 201501) & (regularExporters[COMMODITY] == 'processed')]

,Period,Partner,Trade Flow,Milk and cream,Trade Value (US$)
266,201501,Belgium,Imports,processed,2246219
278,201501,Denmark,Imports,processed,94185
282,201501,France,Imports,processed,4610827
285,201501,Germany,Imports,processed,4557737
296,201501,Ireland,Imports,processed,2412850
298,201501,Italy,Imports,processed,144175
306,201501,Lithuania,Imports,processed,44560
311,201501,Netherlands,Imports,processed,3379355
314,201501,Poland,Imports,processed,457143
316,201501,Portugal,Imports,processed,3144


Where could the export market be further developed, i.e. which countries import the least? Do the figures look realistic?

In [15]:
#exports = milk[milk[FLOW] == 'Exports']
#grouped = exports.groupby([PARTNER])
print('The UK exports to', len(grouped), 'countries.')
print('The 5 smallest importers from the UK are:')
grouped[VALUE].aggregate(sum).order(ascending=True).head()

The UK exports to 107 countries.
The 5 smallest importers from the UK are:


Partner
Croatia       5
Romania      36
Slovakia     65
Estonia     102
Bulgaria    216
Name: Trade Value (US$), dtype: int64

What is total amount of exports to and imports from the bi-lateral trade countries? Hint: pivot tables can have ‘marginal’ values.

In [18]:
countriesWithTotals = pivot_table(milk, index=[PARTNER], columns=[FLOW], 
                        values=VALUE, aggfunc=sum, margins=True, dropna=True)

countriesWithTotals.dropna()

Trade Flow,Exports,Imports,All
Partner,,,
Austria,56279,780,57059
Belgium,7206795,14325697,21532492
Czech Rep.,2590,486628,489218
Denmark,335068,13681759,14016827
France,9371510,28035441,37406951
Germany,8446290,21895561,30341851
Hungary,71378,4762,76140
Ireland,127401424,46161883,173563307
Italy,173687,315677,489364


Repeat the whole analysis for January–May 2014 and compare the results.

## January-May 2014

In [19]:
URL = 'http://comtrade.un.org/api/get?max=5000&type=C&freq=M&px=HS&ps=201405%2C201404%2C201403%2C201402%2C201401&r=826&p=all&rg=1%2C2&cc=0401%2C0402&fmt=csv'
milk2014 = read_csv(URL, dtype={'Commodity Code':str})
milk2014.tail(2)

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Qty,Alt Qty Unit Code,Alt Qty Unit,Alt Qty,Netweight (kg),Gross weight (kg),Trade Value (US$),CIF Trade Value (US$),FOB Trade Value (US$),Flag
604,HS,2014,201405,May 2014,4,0,2,Exports,826,United Kingdom,...,NaN,NaN,NaN,NaN,18480,NaN,30162,NaN,NaN,0
605,HS,2014,201405,May 2014,4,0,2,Exports,826,United Kingdom,...,NaN,NaN,NaN,NaN,50000,NaN,313417,NaN,NaN,0


In [20]:
milk2014[COMMODITY] = milk2014['Commodity Code'].apply(milkType)

milk2014 = milk2014[headings]
milk2014.head()

,Period,Partner,Trade Flow,Milk and cream,Trade Value (US$)
0,201401,World,Imports,unprocessed,21950747
1,201401,World,Exports,unprocessed,46923551
2,201401,Afghanistan,Exports,unprocessed,3410
3,201401,Austria,Exports,unprocessed,316
4,201401,Belgium,Imports,unprocessed,4472349


In [21]:
milk2014 = milk2014[milk2014[PARTNER] != 'World']
milk2014.head()

,Period,Partner,Trade Flow,Milk and cream,Trade Value (US$)
2,201401,Afghanistan,Exports,unprocessed,3410
3,201401,Austria,Exports,unprocessed,316
4,201401,Belgium,Imports,unprocessed,4472349
5,201401,Belgium,Exports,unprocessed,5663128
6,201401,Br. Virgin Isds,Exports,unprocessed,34566


### Total trade flow

In [22]:
grouped2014 = milk2014.groupby([FLOW])
grouped2014[VALUE].aggregate(sum)

Trade Flow
Exports    437806707
Imports    228279328
Name: Trade Value (US$), dtype: int64

### Main trade partners

In [25]:
imports2014 = milk2014[milk2014[FLOW] == 'Imports']
grouped2014 = imports2014.groupby([PARTNER])
print('The UK imports from', len(grouped2014), 'countries in 2014.')
print('The 5 biggest exporters to the UK in 2014 are:')
grouped2014[VALUE].aggregate(sum).sort(inplace=False,ascending=False).head()

The UK imports from 23 countries in 2014.
The 5 biggest exporters to the UK in 2014 are:


Partner
Ireland        75342679
France         43094286
Germany        29234902
Belgium        26762776
Netherlands    25742755
Name: Trade Value (US$), dtype: int64

In [26]:
exports2014 = milk2014[milk2014[FLOW] == 'Exports']
grouped2014 = exports2014.groupby([PARTNER])
print('The UK exports to', len(grouped2014), 'countries in 2014.')
print('The 5 biggest importers from the UK in 2014 are:')
grouped2014[VALUE].aggregate(sum).order(ascending=False).head()

The UK exports to 107 countries in 2014.
The 5 biggest importers from the UK in 2014 are:


Partner
Ireland        200713266
Algeria         56381359
Netherlands     38625707
Belgium         20506014
France          19856738
Name: Trade Value (US$), dtype: int64

### Regular importers

In [27]:
grouped2014 = exports2014.groupby([PARTNER])
regular2014 = grouped2014.filter(buysEveryMonth)
regular2014[(regular2014[MONTH] == 201401) & (regular2014[COMMODITY] == 'processed')]

,Period,Partner,Trade Flow,Milk and cream,Trade Value (US$)
264,201401,Austria,Exports,processed,186
267,201401,Belgium,Exports,processed,812056
271,201401,China,Exports,processed,113950
274,201401,Cyprus,Exports,processed,5301
277,201401,Denmark,Exports,processed,40895
279,201401,Finland,Exports,processed,891
281,201401,France,Exports,processed,189337
284,201401,Germany,Exports,processed,3187636
288,201401,"China, Hong Kong SAR",Exports,processed,1875242
289,201401,Hungary,Exports,processed,814


In [28]:
regular2014[VALUE].sum() / exports2014[VALUE].sum()

0.7245735410809958

Just over 72.45 % of the total UK exports are due to these regular customers.

### Bi-directional trade

In [29]:
countries2014 = pivot_table(milk2014, index=[PARTNER], columns=[FLOW], 
                        values=VALUE, aggfunc=sum)
countries2014.head()

countries2014.dropna()

Trade Flow,Exports,Imports
Partner,,
Belgium,20506014,26762776
Czech Rep.,78055,358822
Denmark,331697,11743192
Finland,721216,12
France,19856738,43094286
Germany,17966838,29234902
Greece,14087,7
Hungary,96600,346
Ireland,200713266,75342679


### Regular exporters

In [31]:
groupedExporters2014 = imports2014.groupby([PARTNER])
regularExporters2014 = groupedExporters2014.filter(buysEveryMonth)
regularExporters2014[(regularExporters2014[MONTH] == 201401) & (regularExporters2014[COMMODITY] == 'processed')]

,Period,Partner,Trade Flow,Milk and cream,Trade Value (US$)
266,201401,Belgium,Imports,processed,3584038
280,201401,France,Imports,processed,8020014
283,201401,Germany,Imports,processed,4545873
291,201401,Ireland,Imports,processed,5966962
293,201401,Italy,Imports,processed,163006
300,201401,Lithuania,Imports,processed,76809
304,201401,Netherlands,Imports,processed,4650774
310,201401,Poland,Imports,processed,277150
323,201401,Sweden,Imports,processed,1162482


### Where could the export market be further developed, i.e. which countries import the least? Do the figures look realistic?

In [32]:
print('The UK exports to', len(grouped2014), 'countries in 2014.')
print('The 5 smallest importers from the UK in 2014 are:')
grouped2014[VALUE].aggregate(sum).order(ascending=True).head()

The UK exports to 107 countries in 2014.
The 5 smallest importers from the UK in 2014 are:


Partner
Croatia      17
Latvia       41
Bulgaria    198
Slovakia    314
Estonia     881
Name: Trade Value (US$), dtype: int64

### What is total amount of exports to and imports from the bi-lateral trade countries? Hint: pivot tables can have ‘marginal’ values.

In [33]:
countriesWithTotals2014 = pivot_table(milk2014, index=[PARTNER], columns=[FLOW], 
                        values=VALUE, aggfunc=sum, margins=True, dropna=True)

countriesWithTotals2014.dropna()

Trade Flow,Exports,Imports,All
Partner,,,
Belgium,20506014,26762776,47268790
Czech Rep.,78055,358822,436877
Denmark,331697,11743192,12074889
Finland,721216,12,721228
France,19856738,43094286,62951024
Germany,17966838,29234902,47201740
Greece,14087,7,14094
Hungary,96600,346,96946
Ireland,200713266,75342679,276055945


## Conclusions

The milk and cream trade of the UK from January to May 2015 was analysed in terms of which countries the UK mostly depends on for income (exports) and goods (imports). Over the period, the UK had a trade surplus of over 100 million US dollars.

Ireland is the main partner, but it imported from the UK almost the triple in value than it exported to the UK. 

The UK exported to over 100 countries during the period, but only imported from 21 countries, the main ones (top five by trade value) being geographically close. China and Hong Kong are the main importers that are not also main exporters. 

The UK is heavily dependent on its regular customers, the 16 countries that buy all types of milk and cream every month. They contribute three quarters of the total export value.

The UK has bi-directional trade (i.e. both exports and imports) with 20 countries, although for some the trade value (in US dollars) is suspiciously low, which raises questions about the data's accuracy.